In [1]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
fake = pd.read_csv("https://confrecordings.ams3.digitaloceanspaces.com/Fake1.csv")
true = pd.read_csv("https://confrecordings.ams3.digitaloceanspaces.com/1_True1.csv")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
print(fake.shape)
print(true.shape)
fake['target'] = 'fake'
true['target'] = 'true'

data = pd.concat([fake, true]).reset_index(drop = True)
from sklearn.utils import shuffle
data = shuffle(data)
data = data.reset_index(drop=True)
data.drop(["date"],axis=1,inplace=True)
data.drop(["title"],axis=1,inplace=True)
data.drop(["subject"],axis=1,inplace=True)
print(data.head())
wordnet = WordNetLemmatizer()
corpus = []

(49, 4)
(49, 4)
                                                text target
0  On Friday, it was revealed that former Milwauk...   fake
1  Donald Trump is afraid of strong, powerful wom...   fake
2  Pope Francis used his annual Christmas Day mes...   fake
3  On Wednesday, Donald Trump took a step no prev...   fake
4  WASHINGTON (Reuters) - Crucial details of the ...   true


In [3]:
def clean_text(text):
    # Cleaning text
    review = re.sub('[^a-zA-Z]', ' ', text)
    # Formating text
    review = review.lower()
    review = review.split()
    # Lemmatize each word in text
    review = [wordnet.lemmatize(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    return review

data["text"] = data["text"].apply(clean_text)
print(data.head())
x = data["text"]
y = data['target']

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics


                                                text target
0  friday revealed former milwaukee sheriff david...   fake
1  donald trump afraid strong powerful woman horr...   fake
2  pope francis used annual christmas day message...   fake
3  wednesday donald trump took step previous pres...   fake
4  washington reuters crucial detail republican t...   true


In [4]:
# Divide the dataset into Train and Test
X_train,X_test,y_train,y_test = train_test_split(x,y, test_size=0.2, random_state=42)

tfidf_v = TfidfVectorizer()
X_tfidf = tfidf_v.fit_transform(X_train)
X_tfidf.shape
x_tfidf = tfidf_v.transform(X_test)
x_tfidf.shape


classifier1 = LogisticRegression(random_state = 0)
classifier1.fit(X_tfidf, y_train)
prediction = classifier1.predict(x_tfidf) # Predict on new data 
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))
cm = metrics.confusion_matrix(y_test, prediction)
print(cm)

accuracy: 85.0%
[[9 0]
 [3 8]]


In [5]:
classifier2 = tree.DecisionTreeClassifier(random_state = 0)
classifier2.fit(X_tfidf, y_train)
prediction = classifier2.predict(x_tfidf) # Predict on new data 
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))
cm = metrics.confusion_matrix(y_test, prediction)
print(cm)


accuracy: 100.0%
[[ 9  0]
 [ 0 11]]


In [ ]:
classifier3=RandomForestClassifier()
classifier3.fit(X_tfidf, y_train)
prediction = classifier3.predict(x_tfidf) # Predict on new data 
print("accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))
cm = metrics.confusion_matrix(y_test, prediction)
print(cm)

print("Train accuracy:", dt1.score(X_tfidf,y_train))

print("Test accuracy:", dt1.score(X_test,y_test))